In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

# The mock data

In [ ]:

# 1. Generate real data
start_time = pd.Timestamp("2025-06-10 00:00:00")
end_time = start_time + pd.Timedelta(days=14)
time_index = pd.date_range(start=start_time, end=end_time, freq="5min")

# simulate real data between 1.6 and 2.1
np.random.seed(42)
real_values = np.random.uniform(1.6, 2.1, len(time_index))

real_df = pd.DataFrame({
    "timestamp": time_index,
    "value": real_values,
    "type": "real"
})

# 2. Generate predicted data for the last 3 days
pred_start_time = end_time - pd.Timedelta(days=7)
pred_time_index = pd.date_range(start=pred_start_time, end=end_time, freq="5min")

# simulate predicted data with some small noise around real data
pred_real_values = real_df.loc[real_df['timestamp'].isin(pred_time_index), "value"].values
pred_noise = np.random.normal(0, 0.05, len(pred_real_values))  # small noise
pred_values = pred_real_values + pred_noise
pred_values = np.clip(pred_values, 1.6, 2.1)  # keep within bounds

pred_df = pd.DataFrame({
    "timestamp": pred_time_index,
    "value": pred_values,
    "type": "predicted"
})




# The plot

In [54]:
# Define predicted data start and end timestamps for the button range
pred_start_time = pred_df.timestamp.min()
pred_end_time = pred_df.timestamp.max()

# Definition of time range for the buttons
buttons = [
    dict(
        label="1 Day",
        method="relayout",
        args=[{
            "xaxis.range": [pred_start_time - pd.Timedelta(days=3), pred_start_time + pd.Timedelta(days=1)]
        }]
    ),
    dict(
        label="3 Days",
        method="relayout",
        args=[{
            "xaxis.range": [pred_start_time - pd.Timedelta(days=3), pred_start_time + pd.Timedelta(days=3)]
        }]
    ),
    dict(
        label="7 Days",
        method="relayout",
        args=[{
            "xaxis.range": [pred_start_time - pd.Timedelta(days=3), pred_start_time + pd.Timedelta(days=7)]
        }]
    ),
    dict(
        label="All",
        method="relayout",
        args=[{
            "xaxis.autorange": True
        }]
    ),
]

# Concatenation of the data is the best way for plotly
plot_df = pd.concat([real_df, pred_df])

# Create the base plot
fig = px.line(
    plot_df,
    x="timestamp",
    y="value",
    color="type",
    labels={"value": "price", "timestamp": "date/time"},
    color_discrete_map={
        "real": "#4281ea",      
        "predicted": "#ff8019"  
    }
)

# Comprehensive layout settings
fig.update_layout(
    # Title settings
    title=dict(
        text="Prediction",
        x=0.5,
        font=dict(size=30, weight='bold')
    ),
    
    # Size and margins
    width=1500,
    height=500,
    margin=dict(l=40, r=40, t=60, b=40),
    
    # Background settings
    template="plotly_white",
    plot_bgcolor="#f9fcfc",
    paper_bgcolor="#f9fcfc",
    
    # X-axis settings
    xaxis=dict(
        title="Time",
        tickfont=dict(size=14),
        tickformatstops=[
            dict(dtickrange=[None, 1000*60*60*6], value="%H:%M"),
            dict(dtickrange=[1000*60*60*6, 1000*60*60*24*2], value="%b %d\n%H:%M"),
            dict(dtickrange=[1000*60*60*24*2, None], value="%b %d"),
        ]
    ),
    
    # Y-axis settings
    yaxis=dict(
        title="e5 price (€/L)",
        title_font=dict(size=25, weight='bold'),
        tickfont=dict(size=20),
        gridcolor="#dcdbd9"
    ),
    
    # Button menu settings
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            buttons=buttons,
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1.15,
            yanchor="top"
        )
    ]
)

# Line style settings
fig.update_traces(line=dict(width=5))

# Display the plot
fig.show()